<a href="https://colab.research.google.com/github/inokchoi/Speech/blob/main/ctc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

히든마커 + 딥러닝 (하이브리드)
딥러닝만은 CTC, LAS

TIMIT - 3개 기관이 만듦, MIT, SRI, TI
6300개 센텐스, 사람 630명, 10개 발음
미국 8개 사투리 모아놓음
DR1, 2만 주셨음 (New England, Northern)
Dialect만 쓰게 될 것입니다.

TIMIT의 PHN 내용을 보면 각각의 segmentation이 되어있다. (마크가 되어있음)
WRD는 워드로 레이블링 되어있다.

RAW, WRD, PHN이 저장이 되어있어서 좋다.

데이터가 Training, test가 같은 것도 있다. (저작권 때문에)

phn_list.txt 보면 맨 앞이 음소(aa) 61개가 있고, 중복된 소리를 묶어서 40개정도만 쓴다. (맨 오른쪽)
 

ipynb: 메인 파일

py파일: 파이썬 파일

phn_list.txt

화자인식처럼 list.txt 있음

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
cd '/content/drive/My Drive/Colab Notebooks/20-1voice_interface/Proj3/'

/content/drive/My Drive/Colab Notebooks/20-1voice_interface/Proj3


In [ ]:
!pip install editdistance
!pip install logger
!pip install tensorboard_logger
!pip install easydict

  Created wheel for logger: filename=logger-1.4-cp36-none-any.whl size=1789 sha256=1d6040cfd06ef4f1fb34be3b8f4b3d712bb649daf4ba8b5979b0be8fc5d024f9
  Stored in directory: /root/.cache/pip/wheels/91/d4/96/08341e2ac92c1ed4b760e4848e1acda3795f0257a83b94b42e
Successfully built logger


In [ ]:
import torch
import pdb
import numpy as np
import sys
import preprocess
import pdb
import os
import editdistance
import time
import easydict
from torch.utils.data import DataLoader
from dataset import *
from model import *
from torch.optim import Adam
from tqdm import tqdm
from logger import Logger

# Store log information in the designated directory.
logger = Logger('./ctc_log/{}'.format(time.ctime()).replace(' ','_').replace(':', '_')) #로그 인포메이션 저장 (트레이닝 중간 결과 저장)

## Decode and LER calculation function

LER: Label error rate 측정하는 것. 길이로 나눠서 구한다.

In [ ]:
def decode(pred, label, target_len, phn_list): #prediction된 결과를 가지고 maximum 찾고, collapse.
    ler = 0
    for i in range(pred.size(1)):
        # Find the maximum likelihood output.
        pred_class = torch.argmax(pred[:,i,:], dim=-1)
        
        # Collapse the repeated outputs. (Note: simplified implementation)
        collapse = [pred_class[k] for k in range(len(pred_class)) if (k==0) or pred_class[k] != pred_class[k-1]]
        
        # Compute the edit distance between the reference and estimated ones.
        collapse = torch.tensor([value for value in collapse if value != 0])
        ed = editdistance.eval(collapse, label[i][:target_len[i]])/target_len[i]  #editdistance 얼마만큼 에러 레이트인지를 볼 수 있게 한다.
        ler += ed
    
    ler /= pred.size(1)
    
    return ler, collapse

## Inference function

In [ ]:
def inference(epoch, model, data_loader, criterion, optimizer, device, 
              phn_list, train=True):
    if train:
        model.train()
        flag = 'Train'
    else:
        model.eval()
        flag='Eval'

    total_loss = 0
    total_ler = 0
    progress_bar = tqdm(enumerate(data_loader)) #프린트가 많아지면 문장이 안보여서 추가함
  
    for b_idx, (data, label, input_lengs, target_lengs) in progress_bar:
        # Move data and label to the device (e.g. GPU).
        data = data.type(torch.FloatTensor).to(device)  
        label = label.type(torch.LongTensor).to(device)  #Label은 integer type이어서
        
        # Predict the output given the input data.
        if train:
            pred = model(data)
        else:
            with torch.no_grad():
                pred = model(data)
                
        # Compute the loss by comparing the predicted output to the reference labels.
        loss = criterion(pred, label, input_lengs, target_lengs)  #CTCLoss
        total_loss += loss.item()
    
        # Decode and calculate LER.
        ler, collapse = decode(pred, label, target_lengs, phn_list) #모델 만든걸 디코드 해서 에러율 알려고
        total_ler += ler
        
        # Perform backpropagation if it is the training mode.
        if train:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        # Display the running progress.
        progress_bar.set_description(
            '{} Epoch:{}[{}/{}] CTC_loss:{:.3f} LER:{:.3f} '.format( \
            flag, epoch, b_idx, len(data_loader), total_loss/(b_idx+1), 
            total_ler/(b_idx+1))
        )
  
    return total_loss, total_ler

## Main function

In [ ]:
def main():
    # Set the environmental parameters.
    args = easydict.EasyDict({
      'train_list':'./timit_train_list.txt',
      'test_list':'./timit_test_list.txt',
      'data_path':'./',  #실제 웨이브파일
      'batch_size':8,
      'input_type':'mfcc',
      'input_dim':13,
      'delta_order':3, #델타 쓸거냐 말꺼냐
      'resume':False, #세이브된것부터 하고 싶을 때
      'resume_model':'ctc_10.pth', #모델 저장 파일
      'lr':0.01,
      'max_epoch':20,
      'log_path':'./logs',
      'save_dir':'./models',       
    })
  
    # Set the device for running the CTC model.
    device  = torch.device('cuda:0')

    # Define a network architecture. ('model.py')
    model = CTC_model(input_dim=args.input_dim*args.delta_order, num_class = 41)
    model = model.to(device) #이걸 안하면 CPU로 한다.

    # Define an optimizer.
    optimizer = Adam(model.parameters(), lr=args.lr)

    # Define a training criterion.
    ctc_criterion = torch.nn.CTCLoss(blank=0)

    # Load the pre-trained model if you resume the training from the model.
    os.makedirs(args.save_dir, exist_ok=True)
    if args.resume:  #이미 수행된 모델이 있으면 모델 읽어서 로드하기.
        if os.path.isfile(os.path.join(args.save_dir, args.resume_model)):
            checkpoint = torch.load(os.path.join(args.save_dir, args.resume_model),map_location=device)
            model.load_state_dict(checkpoint['encoder'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            start = checkpoint['epoch']
            print('----loading checkpoint----')
        else:
            print('can not find the resume model.')
    else:
        start=0

    # Set the phone list for decoding. ('preprocess.py')
    phn_list = preprocess.phn_list() #61리스트 가져온다. 3개 열 중 맨 뒤에꺼만 가져온다. 코드 보면 리스트를 set으로 같은건 만든다.

    # Define training and test datasets. ('dataset.py')
    train_dataset = ASR_Dataset(args.train_list, args.data_path, d_type=args.input_type, #mel, mfcc쓸거냐 get feature하는 것, collat 잘 하면 문제 없을 것이다.
                                coeff=args.input_dim, delta=args.delta_order)
    train_loader = DataLoader(train_dataset, #화자인식 때 DataLoader 했음, 영상도 어떻게 feeding할지만 만들어놓으면 쉬움
                              batch_size = args.batch_size,
                              collate_fn = collate_fn,
                              shuffle=True,
                              num_workers=4,
                              pin_memory=True)

    test_dataset = ASR_Dataset(args.test_list, args.data_path, d_type=args.input_type,
                               coeff=args.input_dim, delta=args.delta_order)
    test_loader = DataLoader(test_dataset,
                             batch_size=1,
                             collate_fn = collate_fn,
                             shuffle=False,
                             num_workers=4,
                             pin_memory=True)

    # Perform training/validation processing.
    for epoch in range(start, args.max_epoch+1):
        train_loss, train_ler = inference(epoch, model, train_loader, ctc_criterion, 
                                          optimizer, device, phn_list, train=True)
        valid_loss, valid_ler = inference(epoch, model, test_loader, ctc_criterion, 
                                          optimizer, device, phn_list, train=False)
        
        # Save the trained model at every 10 epochs.
        if epoch % 10 == 0:
            torch.save({'epoch':epoch, 'encoder':model.state_dict(),
                        'optimizer':optimizer.state_dict()},
                        '{}/ctc_{}.pth'.format(args.save_dir, epoch))

        # Log the loss and LER.
        logger.log_value('train_loss', train_loss)
        logger.log_value('train_ler', train_ler)
        logger.log_value('valid_loss', valid_loss)
        logger.log_value('valid_ler', valid_ler)
        logger.step()

In [ ]:
# Run the main functoin.
if __name__ == '__main__':
    main()

Train Epoch:0[113/114] CTC_loss:3.970 LER:0.984 : : 114it [06:01,  3.17s/it]
Eval Epoch:0[219/295] CTC_loss:3.301 LER:0.969 : : 219it [01:57,  1.03s/it]

KeyboardInterrupt: ignored

LER이 1을 넘을 수도 있다.
20프로 정도 됐을 때 트레이닝이 잘 됐다고 할 수 있다.

mel 줄이는게 차수를 줄이는게 더 효율적이다.
데이터 많으면 mel filter 안쓰고 linear 써도 된다.
적은 데이터로는 mel 쓴다.

mel -- log mel filter bank의 dct를 하게 되면 그게 mfcc가 된다.
log mel은 주파수 도메인 에너지
mfcc는 한번 더 푸리에 해서 캡스럴 콘시트가 ?
각각 엘리먼트가 인디펜던트하게 나오게 된다.
데이터 적을 때 더 효과적이다.

stft변환 시 2의 n승을 쓰는게 가까운게 보편적.
n+1은 정보차는 없지만 분해능은 더 좋다.